In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir(r'/content/drive/MyDrive/Cody - AIMI 2024/2024 AIMI Summer Internship - Intern Materials/Datasets')

In [3]:
!unzip -qq student_data_split.zip -d /content/

In [4]:
# Switch back to /content/student_data_split folder to work with downloaded datasets
os.chdir(r'/content/student_data_split')

In [5]:
# Confirm we can now see the student_test and student_train folders + Reports.json
!ls

Reports.json  student_test  student_train


In [6]:
%%capture
%pip install "comet_ml>=3.38.0" torch torchvision tqdm
%pip install timm
%pip install scikit-multilearn

In [7]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import watch


In [8]:
import torch
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm
from torchvision.transforms import v2
from torch import nn
from torchvision import models

import timm

from skmultilearn.model_selection import iterative_train_test_split
from torch.utils.data import DataLoader

In [9]:
dataframe = pd.read_pickle("/content/drive/MyDrive/Cody - AIMI 2024/train_data.pkl")

In [10]:
dataframe

,Patient ID,Study ID,Image Path,Label,Encoded Labels
0,patient39668,student_train/patient39668/study2,student_train/patient39668/study2/view1_fronta...,normal,"[0.0, 0.0, 0.0, 1.0]"
1,patient17014,student_train/patient17014/study2,student_train/patient17014/study2/view1_fronta...,pneumothorax,"[0.0, 1.0, 0.0, 0.0]"
2,patient11443,student_train/patient11443/study1,student_train/patient11443/study1/view1_fronta...,pneumothorax,"[0.0, 1.0, 0.0, 0.0]"
3,patient29294,student_train/patient29294/study1,student_train/patient29294/study1/view1_fronta...,"pneumothorax, pleural effusion","[0.0, 1.0, 1.0, 0.0]"
4,patient34615,student_train/patient34615/study71,student_train/patient34615/study71/view1_front...,pleural effusion,"[0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...
16767,patient33560,student_train/patient33560/study1,student_train/patient33560/study1/view1_fronta...,"pneumonia, pleural effusion","[1.0, 0.0, 1.0, 0.0]"
16768,patient33560,student_train/patient33560/study1,student_train/patient33560/study1/view2_latera...,"pneumonia, pleural effusion","[1.0, 0.0, 1.0, 0.0]"
16769,patient29285,student_train/patient29285/study1,student_train/patient29285/study1/view1_fronta...,"pneumothorax, pleural effusion","[0.0, 1.0, 1.0, 0.0]"
16770,patient29285,student_train/patient29285/study1,student_train/patient29285/study1/view2_latera...,"pneumothorax, pleural effusion","[0.0, 1.0, 1.0, 0.0]"


In [11]:
from torch.utils.data import Dataset

class ChestXRayDataset(Dataset):
    def __init__(self, dataframe, transforms):
        self.dataframe = dataframe
        self.transforms = transforms

    def __len__(self):
        return len(self.dataframe)


    def __getitem__(self, idx):
        out_dict = {"idx": torch.tensor(idx),}

        image_path = self.dataframe.loc[idx,'Image Path']
        labels = self.dataframe.loc[idx,'Encoded Labels']

        image = Image.open(image_path).convert("RGB")
        if(self.transforms is not None):
            image = self.transforms(image)

        out_dict["img"] = image
        out_dict["label"] = torch.tensor(labels, dtype=torch.float32)
        return out_dict["img"], out_dict["label"]


In [12]:
class Resnext50(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        resnet = models.resnext50_32x4d(pretrained=True)
        resnet.fc = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=resnet.fc.in_features, out_features=n_classes)
        )
        self.base_model = resnet
        self.sigm = nn.Sigmoid()

    def forward(self, x):
        return self.sigm(self.base_model(x))

## Define Training Components
Here, define any necessary components that you need to train your model, such as the model architecture, the loss function, and the optimizer.

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
#hyperparameters
batch_size = 64
learning_rate = 1e-4

num_epochs = 20
save_every_x_epochs = 2
current_epoch = 0

In [15]:
X_image_features = dataframe['Image Path'].to_numpy().reshape(-1, 1)
Y_labels = np.vstack(dataframe['Encoded Labels'])
X_image_train, y_labels_train, X_image_test, y_labels_test = iterative_train_test_split(X_image_features, Y_labels, test_size=0.20)

train_df = pd.DataFrame({'Image Path': X_image_train.flatten(), 'Encoded Labels':[l.tolist() for l in y_labels_train]})
val_df = pd.DataFrame({'Image Path': X_image_test.flatten(), 'Encoded Labels':[l.tolist() for l in y_labels_test]})

In [16]:
transforms = v2.Compose([
    v2.ToImage(),
    v2.Resize(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = ChestXRayDataset(train_df, transforms)
val_dataset = ChestXRayDataset(val_df, transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


num_classes = 4
label_space = ['pneumonia', 'pneumothorax', 'pleural effusion', 'normal']

In [17]:
loss_fn = torch.nn.BCEWithLogitsLoss()

model = Resnext50(num_classes)
model.train()
model.to(device)
opt = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, num_epochs)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:01<00:00, 63.2MB/s]


In [18]:
experiment = Experiment(
  api_key="REDACTED",
  project_name="aimi2024-resnext50-no_kfolds",
  workspace="summit"
)
watch(model)


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/summit/aimi2024-resnext50-no-kfolds/e3f19fd1be6f4867ab327ad9b4c251a7

COMET INFO: Couldn't find a Git repository in '/content/student_data_split' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


## Training Code
We provide starter code below that implements a simple training loop in PyTorch. Feel free to modify as you see fit.

In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix, roc_auc_score

def calculate_metrics(pred, target, threshold=0.5):
    thresholded_preds = np.empty_like(pred)
    thresholded_preds[:] = pred
    thresholded_preds = np.array(thresholded_preds > threshold, dtype=float)

    f1 = f1_score(y_true=target, y_pred=thresholded_preds, average=None)
    f1_macro = f1_score(y_true=target, y_pred=thresholded_preds, average='macro')

    auc = roc_auc_score(y_true=target, y_score=pred, average=None)
    auc_macro = roc_auc_score(y_true=target, y_score=pred, average='macro')

    return {'f1': f1, 'f1_macro': f1_macro, 'auc': auc, 'auc_macro': auc_macro}

In [20]:
def train_one_epoch():
      train_loss = 0.0

      for index, (inputs, targets) in enumerate(tqdm(train_loader)):
        inputs, targets = inputs.to(device), targets.to(device)

        opt.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, targets)
        loss.backward()
        opt.step()

        train_loss += loss.item() * inputs.size(0)

      train_loss /= len(train_loader.dataset)
      return train_loss

In [21]:
threshold = 0.5
best_macro_f1 = 0.0 #higher f1 score is better, 1 is best

trainingEpoch_loss = []
validationEpoch_loss = []

In [ ]:
# Epoch loop
for epoch in range(0, num_epochs):
  current_epoch += 1
  print(f"Training epoch {current_epoch}")

  model.train()
  train_loss = train_one_epoch()
  val_loss = 0.0

  print(f"Evaluating...")
  model.eval()

  with torch.no_grad():
    total_results = []
    total_targets = []

    for index, (data, target) in enumerate(tqdm(val_loader)):
        data, target = data.to(device), target.to(device)
        output = model(data)

        total_results.extend(output.cpu().numpy())
        total_targets.extend(target.cpu().numpy())

        val_loss += loss_fn(output, target).item() * target.size(0)

  scheduler.step()

  val_loss /= len(val_loader.dataset)

  trainingEpoch_loss.append(train_loss)
  validationEpoch_loss.append(val_loss)

  metrics = calculate_metrics(np.array(total_results), np.array(total_targets), threshold=threshold)
  print(f'LOSS: train {train_loss} valid {val_loss}')
  print(f'Macro F1 Score: {metrics["f1_macro"]}   Class Breakdown: {metrics["f1"]}')
  print(f'Macro AUROC: {metrics["auc_macro"]}   Class Breakdown: {metrics["auc"]}')

  #Log to Comet
  experiment.log_metric("train loss", train_loss, epoch=current_epoch)
  experiment.log_metric("val loss", val_loss, epoch=current_epoch)
  experiment.log_metric("macro f1", metrics["f1_macro"], epoch=current_epoch)
  experiment.log_metric("macro auroc", metrics["auc_macro"], epoch=current_epoch)

  #Save best checkpoint based on F1 Score
  #F1 Score preferred over AUROC b/c of data imbalance - https://stackoverflow.com/questions/44172162/f1-score-vs-roc-auc
  if(metrics["f1_macro"] > best_macro_f1):
      best_macro_f1 = metrics["f1_macro"]
      state = {
          'epoch': current_epoch,
          'state_dict': model.state_dict(),
          'optimizer': opt.state_dict(),
      }
      save_path = f'/content/drive/MyDrive/Cody - AIMI 2024/Trains/IN_PROGRESS_ResNext50/best.ckpt'
      torch.save(state, save_path)

  #Save every x epochs
  if(current_epoch % save_every_x_epochs == 0):
      save_path = f'/content/drive/MyDrive/Cody - AIMI 2024/Trains/IN_PROGRESS_ResNext50/model-epoch-{current_epoch}.pth'
      torch.save(model.state_dict(), save_path)  # Save model weights for inference


Training epoch 1


100%|██████████| 210/210 [05:06<00:00,  1.46s/it]


Evaluating...


100%|██████████| 53/53 [00:44<00:00,  1.18it/s]


LOSS: train 0.6926052004405309 valid 0.681797157437054
Macro F1 Score: 0.20242649357296055   Class Breakdown: [0.         0.66954753 0.14015844 0.        ]
Macro AUROC: 0.6049183314487818   Class Breakdown: [0.65325132 0.64705131 0.58841759 0.53095311]
Training epoch 2


...

100%|██████████| 210/210 [04:50<00:00,  1.38s/it]


Evaluating...


100%|██████████| 53/53 [00:36<00:00,  1.44it/s]


LOSS: train 0.6374867066436793 valid 0.6785974664474601
Macro F1 Score: 0.25558697574826605   Class Breakdown: [0.         0.68412698 0.33822092 0.        ]
Macro AUROC: 0.573348915845735   Class Breakdown: [0.49995867 0.684713   0.59510081 0.51362319]
Training epoch 15


 55%|█████▍    | 115/210 [02:43<02:06,  1.33s/it]

In [ ]:
#Save last ckpt
state = {
    'epoch': current_epoch,
    'state_dict': model.state_dict(),
    'optimizer': opt.state_dict(),
}
save_path = f'/content/drive/MyDrive/Cody - AIMI 2024/Trains/IN_PROGRESS_ResNext50/last.ckpt'
torch.save(state, save_path)

In [ ]:
experiment.end()

# Loss curves

In [ ]:
from matplotlib import pyplot as plt
plt.plot(trainingEpoch_loss, label='train_loss')
plt.plot(validationEpoch_loss,label='val_loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(np.arange(len(trainingEpoch_loss)), np.arange(1, len(trainingEpoch_loss)+1))
plt.legend()

plt.savefig('/content/drive/MyDrive/Cody - AIMI 2024/Trains/IN_PROGRESS_ResNext50/train_val_loss.png')
plt.show()

# Evaluating on Test Dataset


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Resnext50(num_classes)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

checkpoint = torch.load(r'/content/drive/MyDrive/Cody - AIMI 2024/Trains/IN_PROGRESS_ResNext50/best.ckpt')
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
#model.load_state_dict(checkpoint)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
test_transforms = v2.Compose([
    v2.ToImage(),
    v2.Resize(size=(224, 224), antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def getImage(image_path):
  image = Image.open(image_path).convert("RGB")
  image = test_transforms(image)
  return image

In [ ]:
threshold = 0.5

def predict(model, image_tensor):
  model.eval()
  with torch.no_grad():
      input = image_tensor.unsqueeze(0)  #image lacks batch layer, so insert a batch dimension of size 1
      input = input.to(device)

      outputs = model(input).cpu()
      preds = np.array(outputs)

      rounded_preds, thresholded_preds = np.empty_like(preds), np.empty_like(preds)
      rounded_preds[:] = preds
      thresholded_preds[:] = preds

      for(i, pred) in enumerate(preds):
              thresholded_preds[i] = (pred > threshold).astype(int)
              if(np.all(thresholded_preds[i] == 0)):
                thresholded_preds[i] = [0, 0, 0, 1]
              rounded_preds[i] = [round(num, 7) for num in pred]

      return thresholded_preds[0], rounded_preds[0]


In [ ]:
test_dataframe = pd.read_csv("/content/drive/MyDrive/Cody - AIMI 2024/2024 AIMI Summer Internship - Intern Materials/Datasets/test_annotations.csv")
os.chdir(r'/content/student_data_split')

In [ ]:
#np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

processed_patients = []
#number_pneumonia = 0 #temporary just to double check

for index, row in tqdm(test_dataframe.iterrows(), total=test_dataframe.shape[0]):
  images = os.listdir(row['study_id'])
  for image in images:
    thresholded_preds, rounded_preds = predict(model, getImage(row['study_id'] + '/' + image))

    patient = {
        'study_id' : row['study_id'],
        'Pneumothorax' : thresholded_preds[1],
        'Pneumonia' : thresholded_preds[0],
        'Pleural Effusion' : thresholded_preds[2],
        'No Finding' : thresholded_preds[3],
        'Pneumothorax Probs' : rounded_preds[1],
        'Pneumonia Probs' : rounded_preds[0],
        'Pleural Effusion Probs' : rounded_preds[2],
        'No Finding Probs' : rounded_preds[3],
    }

    #temporary, check # of pneumonia to make sure not exporting wrong
    #if(thresholded_preds[0] == 1):
    #  number_pneumonia += 1

    processed_patients.append(patient)

    break # too lazy to deal/combine output from multiple images for now, will handle later

#print(f"\n {number_pneumonia} pneumonia detected")

100%|██████████| 2983/2983 [00:56<00:00, 52.43it/s]


In [ ]:
test_processed_dataframe = pd.DataFrame(processed_patients)
test_processed_dataframe.to_csv(r'/content/drive/MyDrive/Cody - AIMI 2024/test_results.csv', index=False, float_format='%.10f')

# Submitting Your Results
Once you have successfully trained your model, generate predictions on the test set and save your results as a `.csv` file. This file can then be uploaded to the leaderboard: https://vilmedic.app/misc/aimi24/leaderboard.

An example `test_results.csv` has been provided for reference only in the `2024 AIMI Summer Internship - Intern Materials/Datasets/Labels` folder. *Do not submit this, the results will be really poor. *

Your final `.csv` file **must** have the following format:
- There must be a column titled `study_id` with the paths to the study_id for the test set image, e.g. `student_test/patient35172/study3`.
- The provided columns from `test_annotations.csv` must be present: "Pneumothorax", "Pneumonia", "Pleural Effusion", "No Finding:
  - Each of these columns must contain a binary value `0` or `1` representing the **observed/ground-truth** absence or presence of the disease status.
- Added columns "Pneumothorax Probs", "Pneumonia Probs", "Pleural Effusion Probs", "No Finding Probs" containing the singular probability values belonging to each class.
  - Each of these columns must contain a continuous value representing the **predicted** probability of the absence or presence of the disease status for that class.
  - *Hint:* Depending on which loss function you used, you might already be outputing probabilities. You can then derive predictions by thresholding your probabilities to a binarized output. If your model outputs logits directly, then apply the sigmoid activation function `torch.sigmoid(logits)` to get probabilities and then threshold to get binary predictions.
- Double check that the length of the dataset passed into your dataloader matches the length of your final dataframe.

In [ ]:
model = # Model Architecture
ckpt = torch.load("/content/best.pkl")
model.load_state_dict(ckpt["state_dict"])

test_dataset = ChestXRayDataset("""Fill in args here""")
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=4, shuffle=False, drop_last=False)

In [ ]:
# Write method to load in data from test_loader, compute model predictions, and append results to test_results dict
test_results = {"image_path": [], "pred": []}

In [ ]:
test_results = pd.DataFrame(test_results)
test_results.to_csv(f"/content/test_results.csv")